In [1]:
import hashlib
import base58
import hashlib
import requests
import csv
import os
import warnings
import gc
import argparse
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import ast
import re
import time
import csv
import hashlib
import os
import psutil
import csv
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import hashlib
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import timedelta

# Suppress all warnings
warnings.filterwarnings("ignore")

import logging

# Set up logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)  # Set the logging level

# Remove any default handlers associated with the logger
logger.handlers = []

# Create a file handler
handler = logging.FileHandler('worklog.log', encoding='utf-8')
# Create a rotating file handler
handler.setLevel(logging.DEBUG)  # Set the handler level

# Create a logging format
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Add the file handler to the logger
logger.addHandler(handler)

# Disable propagation to avoid printing to the console
logger.propagate = False

In [2]:
import requests
import datetime
TRON_NODE_URL = "https://trx-rpc.transatron.io"

def get_block_transactions(block_number):
    url = f"{TRON_NODE_URL}/wallet/getblockbynum"
    response = requests.post(url, json={"num": block_number})
    response.raise_for_status()
    block_data = response.json()
    transactions = block_data.get("transactions", [])
    for tx in transactions:
        tx["block_timestamp"] = block_data["block_header"]["raw_data"]["timestamp"]
    return transactions

In [3]:
# Cache for addresses to speed up conversion
address_cache = {}

def hex_to_base58(hex_address):
    """
    Converts a Tron address from hex format to Base58 format.
    """
    if hex_address in address_cache:
        return address_cache[hex_address]
    if not hex_address.startswith("41"):
        raise ValueError("Invalid Tron address: Must start with '41'")

    address_bytes = bytes.fromhex(hex_address)
    checksum = hashlib.sha256(hashlib.sha256(address_bytes).digest()).digest()[:4]
    address_with_checksum = address_bytes + checksum
    base58_address = base58.b58encode(address_with_checksum).decode()
    address_cache[hex_address] = base58_address
    return base58_address

def save_to_csv(transactions, output_dir, block_number):
    """
    Saves filtered transactions to CSV files organized by month.
    Adds address conversion to TRON standard in the CSV.

    Args:
        transactions (list): List of transaction dictionaries.
        output_dir (str): Directory to save the CSV files.
        block_number (str): Block number for file name.
    """
    files = {}
    try:
        for tx in transactions:
            timestamp = tx.get("block_timestamp")
            if not timestamp:
                continue  # Skip transactions without timestamp
            month = datetime.datetime.fromtimestamp(timestamp / 1000).strftime("%Y-%m")

            # Prepare the output file and writer
            if month not in files:
                output_file = os.path.join(output_dir, f"transactions-block-{block_number}.csv")
                file_exists = os.path.isfile(output_file)
                csvfile = open(output_file, mode="a", newline="", encoding="utf-8")

                # Define the fieldnames (columns)
                fieldnames = [
                    "txID", "signature", "contractRet", "ref_block_bytes", "ref_block_hash", "block_number",
                    "timestamp", "owner_address_1", "owner_weight_1", "owner_address_2", "owner_weight_2",
                    "actives_address_1", "actives_weight_1", "actives_address_2", "actives_weight_2",
                    "owner_address", "owner_address_1_tron", "owner_address_2_tron", "actives_address_1_tron", "actives_address_2_tron", "block_timestamp",
                    "delegate_reclaim_after_uap_count", "stake_count", "vote_count", "transfer_count", "is_transfer_to_the_second_owner", "additional_uap"
                ]
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                if not file_exists:
                    writer.writeheader()
                files[month] = (csvfile, writer)

            # Extract the transaction data
            txID = tx.get('txID', '')
            signature = tx.get('signature', [''])[0]
            contractRet = tx.get('ret', [{}])[0].get('contractRet', '')
            ref_block_bytes = tx.get('raw_data', {}).get('ref_block_bytes', '')
            ref_block_hash = tx.get('raw_data', {}).get('ref_block_hash', '')
            block_timestamp = datetime.datetime.fromtimestamp(tx.get('block_timestamp', 0) / 1000).isoformat()

            # Extract values from the contract
            contract = tx.get('raw_data', {}).get('contract', [])
            for item in contract:
                parameter = item.get('parameter', {}).get('value', {})

                # Extract 'owner' data
                if 'owner' in parameter:
                    owner_permission = parameter['owner']
                    addresses = owner_permission.get('keys', [])

                    # Handle missing second address by checking list length
                    owner_address_1 = addresses[0].get('address', '')
                    owner_weight_1 = addresses[0].get('weight', '')
                    if len(addresses) > 1:
                        owner_address_2 = addresses[1].get('address', '')
                        owner_weight_2 = addresses[1].get('weight', '')
                    else:
                        owner_address_2 = ''
                        owner_weight_2 = ''

                    owner_address = parameter.get('owner_address', '')

                    # Extract 'actives' data
                    actives_address_1 = actives_weight_1 = actives_address_2 = actives_weight_2 = ''
                    if 'actives' in parameter:
                        actives = parameter['actives']
                        actives_address_1 = actives[0]['keys'][0].get('address', '')
                        actives_weight_1 = actives[0]['keys'][0].get('weight', '')
                        if len(actives[0]['keys']) > 1:
                            actives_address_2 = actives[0]['keys'][1].get('address', '')
                            actives_weight_2 = actives[0]['keys'][1].get('weight', '')
                        else:
                            actives_address_2 = ''
                            actives_weight_2 = ''

                    # Convert addresses to TRON standard (Base58 format)
                    owner_address_1_tron = hex_to_base58(owner_address_1)
                    owner_address_2_tron = hex_to_base58(owner_address_2) if owner_address_2 else ''
                    actives_address_1_tron = hex_to_base58(actives_address_1)
                    actives_address_2_tron = hex_to_base58(actives_address_2) if actives_address_2 else ''

                    # Write the extracted data to the CSV file
                    writer = files[month][1]
                    writer.writerow({
                        "txID": txID,
                        "signature": signature,
                        "contractRet": contractRet,
                        "ref_block_bytes": ref_block_bytes,
                        "ref_block_hash": ref_block_hash,
                        "block_number": block_number,
                        "timestamp": datetime.datetime.fromtimestamp(timestamp / 1000).isoformat(),
                        "owner_address_1": owner_address_1,
                        "owner_weight_1": owner_weight_1,
                        "owner_address_2": owner_address_2,
                        "owner_weight_2": owner_weight_2,
                        "actives_address_1": actives_address_1,
                        "actives_weight_1": actives_weight_1,
                        "actives_address_2": actives_address_2,
                        "actives_weight_2": actives_weight_2,
                        "owner_address": owner_address,
                        "owner_address_1_tron": owner_address_1_tron,
                        "owner_address_2_tron": owner_address_2_tron,
                        "actives_address_1_tron": actives_address_1_tron,
                        "actives_address_2_tron": actives_address_2_tron,
                        "block_timestamp": block_timestamp,
                        "delegate_reclaim_after_uap_count": 0,  # Assuming constant field
                        "stake_count": 0,  # Assuming constant field
                        "vote_count": 0,  # Assuming constant field
                        "transfer_count": 0,  # Assuming constant field
                        "is_transfer_to_the_second_owner": False,  # Assuming constant field
                        "additional_uap": 0  # Assuming constant field
                    })

    finally:
        # Close all open files
        for csvfile, _ in files.values():
            csvfile.close()

    #print("CSV file has been written successfully.")

In [4]:
def concat_and_save_to_csv(output_dir, existing_csv_name, block_number):
    """
    Concatenates the new block's CSV data to the existing general CSV (UAP transactions).
    The new block's data is identified by the block_number and saved to the same output file.

    Args:
        output_dir (str): Directory where the CSV files are located.
        existing_csv_name (str): Filename of the existing general CSV (UAP transactions).
        block_number (str): The current block number to identify the CSV file for the new block.

    Returns:
        pd.DataFrame: A DataFrame containing the concatenated data.
    """
    # Construct the paths for the existing CSV and the new CSV based on block_number
    existing_csv = os.path.join(output_dir, existing_csv_name)
    new_csv = os.path.join(output_dir, f"transactions-block-{block_number}.csv")

    # Load the existing general CSV
    if os.path.exists(existing_csv):
        existing_df = pd.read_csv(existing_csv)
    else:
        # If the general CSV doesn't exist, initialize as empty DataFrame
        #print(f"Warning: {existing_csv} not found. Will create a new file with the current data.")
        existing_df = pd.DataFrame()

    # Load the new block's CSV data (based on block_number)
    if os.path.exists(new_csv):
        new_df = pd.read_csv(new_csv)
    else:
        #rint(f"Warning: {new_csv} not found. Returning the existing data.")
        new_df = pd.DataFrame()

    # If existing_df is empty, just save the new data
    if existing_df.empty:
        combined_df = new_df
    else:
        # Concatenate the existing and new DataFrames
        combined_df = pd.concat([existing_df, new_df], ignore_index=True)

        # Remove duplicates based on 'owner_address', keeping the last one
        #combined_df = combined_df.drop_duplicates(subset='owner_address', keep='first')

        # Optionally, here you can add logic to modify or update rows for the same `owner_address`
        # For example, here you can apply updates like changing `stake_count`, etc.

        # Example update: Let's add 1 to `stake_count` for all rows with the same `owner_address`
        #combined_df['stake_count'] = combined_df.groupby('owner_address')['stake_count'].transform(lambda x: x + 1)

    # Save the updated DataFrame back to the existing general CSV
    combined_df.to_csv(existing_csv, index=False)

    #print(f"Combined data saved to {existing_csv}")

    return combined_df

In [5]:
def delete_block_csv(output_dir, block_number):
    """
    Deletes the CSV file generated for a particular block from the directory.

    Args:
        output_dir (str): Directory where the CSV files are located.
        block_number (str): Block number to identify the CSV file to delete.

    Returns:
        None
    """
    # Construct the path for the block CSV file based on block_number
    block_csv = os.path.join(output_dir, f"transactions-block-{block_number}.csv")

    # Check if the file exists and delete it
    if os.path.exists(block_csv):
        os.remove(block_csv)
        #print(f"Deleted {block_csv} successfully.")
    #else:
        #print(f"File {block_csv} does not exist. No action taken.")

In [6]:
block_number=68323882

agg_file_dir='UAP_agg.csv'
block_last=block_number

get_block_temp=get_block_transactions(block_number)
save_to_csv(get_block_temp, '', block_number)
#concat_and_save_to_csv('', agg_file_dir, block_number)
#update_additional_uap_from_transactions(get_block_temp, agg_file_dir)
#update_delegate_reclaim_count_from_transactions(get_block_temp, agg_file_dir)
#update_stake_count_from_transactions(get_block_temp, agg_file_dir)
#update_uap_agg_from_transfers(get_block_temp, agg_file_dir)
#filter_invalid_rows_and_save(agg_file_dir)
#delete_block_csv('', block_number)

In [10]:
def process_block(block_number):
    """
    Process a single block:
    - Get block transactions
    - Save to CSV
    - Concatenate with existing data
    - Delete the temporary block CSV file
    """
    get_block_temp = get_block_transactions(block_number)
    save_to_csv(get_block_temp, '', block_number)
    #concat_and_save_to_csv('', agg_file_dir, block_number)
    #delete_block_csv('', block_number)

In [9]:
def aggregate_and_cleanup_csvs(output_filename="aggregated_transactions.csv"):
    # Get a list of all files in the current directory starting with "transaction"
    transaction_files = [file for file in os.listdir() if file.startswith("transaction") and file.endswith(".csv")]
    
    if not transaction_files:
        print("No transaction files found.")
        return

    # List to hold dataframes
    dataframes = []

    # Read each transaction file into a DataFrame and append to the list
    for file in transaction_files:
        try:
            df = pd.read_csv(file)
            dataframes.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")

    # Concatenate all dataframes into one
    if dataframes:
        aggregated_df = pd.concat(dataframes, ignore_index=True)

        # Save the aggregated dataframe to a CSV
        aggregated_df.to_csv(output_filename, index=False)
        print(f"Aggregated file saved as {output_filename}")

        # Delete the individual transaction files
        for file in transaction_files:
            try:
                os.remove(file)
                print(f"Deleted {file}")
            except Exception as e:
                print(f"Error deleting {file}: {e}")
    else:
        print("No dataframes to aggregate.")

In [11]:
def aggregate_and_cleanup_csvs(output_filename="aggregated_transactions.csv"):
    """
    Aggregates all CSV files in the current directory that start with 'transaction'
    into one CSV file and deletes the individual files.

    Args:
        output_filename (str): The name of the output file for the aggregated data.
    
    Returns:
        dict: A dictionary with keys 'aggregated', 'deleted', and 'errors', containing:
              - 'aggregated': True if the aggregation was successful, False otherwise.
              - 'deleted': A list of files successfully deleted.
              - 'errors': A list of errors encountered during the process.
    """
    result = {
        "aggregated": False,
        "deleted": [],
        "errors": []
    }

    # Get a list of all files in the current directory starting with "transaction"
    transaction_files = [file for file in os.listdir() if file.startswith("transaction") and file.endswith(".csv")]

    if not transaction_files:
        return result

    # List to hold dataframes
    dataframes = []

    # Read each transaction file into a DataFrame and append to the list
    for file in transaction_files:
        try:
            df = pd.read_csv(file)
            dataframes.append(df)
        except Exception as e:
            result["errors"].append({"file": file, "error": str(e)})

    # Concatenate all dataframes into one
    if dataframes:
        aggregated_df = pd.concat(dataframes, ignore_index=True)

        try:
            # Save the aggregated dataframe to a CSV
            aggregated_df.to_csv(output_filename, index=False)
            result["aggregated"] = True
        except Exception as e:
            result["errors"].append({"file": output_filename, "error": str(e)})
            return result

        # Delete the individual transaction files
        for file in transaction_files:
            try:
                os.remove(file)
                result["deleted"].append(file)
            except Exception as e:
                result["errors"].append({"file": file, "error": str(e)})

In [12]:
def process_batch(block_last, batch_size):
    
        # Process blocks in parallel using ThreadPoolExecutor
        block_last = block_last  # Adjust to your starting block
        block_numbers = range(block_last, block_last + batch_size)  # Adjust the range of blocks you want to process

        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_block = {executor.submit(process_block, block_number): block_number for block_number in block_numbers}

            # Track progress with tqdm for parallel tasks
            for future in tqdm(as_completed(future_to_block), total=len(block_numbers), desc="Processing blocks", unit="block"):
                block_number = future_to_block[future]
                try:
                    future.result()  # Wait for the block processing to finish
                except Exception as e:
                    continue
                    #print(f"Block {block_number} generated an error: {e}")
                    
        aggregate_and_cleanup_csvs(f"aggregated_transactions-{block_last}.csv")

In [16]:
batch_size = 10000
block_last = 65700000
while block_last < 69200000:

    process_batch(block_last, batch_size)
    block_last =  block_last + batch_size



Processing blocks: 100%|██████████| 10000/10000 [00:57<00:00, 174.49block/s]


In [14]:
# Конфігурація для багатопотокової обробки
batch_size = 100  # наприклад, розмір пачки
offset = 0  # початковий зсув
max_workers = 3  # кількість потоків

In [15]:
# Get the number of logical CPU cores
num_cores = os.cpu_count()

# Get available memory in MB
available_memory = psutil.virtual_memory().available / (1024 * 1024)

# Estimate the memory usage per thread (in MB)
memory_per_thread = 100  # Adjust based on your task's requirements
# Calculate max workers based on memory
max_workers_memory = int(available_memory // memory_per_thread)

# Choose the smaller value between CPU-bound and memory-bound limits
max_workers = min(num_cores * 2, max_workers_memory)  # Assuming I/O-bound tasks
print(f"Optimal max_workers: {max_workers}")

Optimal max_workers: 24


In [14]:
# Process blocks in parallel using ThreadPoolExecutor
block_last = 66000000  # Adjust to your starting block
block_numbers = range(block_last, block_last + 1000)  # Adjust the range of blocks you want to process

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    future_to_block = {executor.submit(process_block, block_number): block_number for block_number in block_numbers}

    # Track progress with tqdm for parallel tasks
    for future in tqdm(as_completed(future_to_block), total=len(block_numbers), desc="Processing blocks", unit="block"):
        block_number = future_to_block[future]
        try:
            future.result()  # Wait for the block processing to finish
        except Exception as e:
            continue
            #print(f"Block {block_number} generated an error: {e}")

Processing blocks: 100%|██████████| 1000/1000 [00:15<00:00, 63.11block/s]


In [16]:
def aggregate_and_cleanup_csvs(output_filename="aggregated_transactions.csv"):
    # Get a list of all files in the current directory starting with "transaction"
    transaction_files = [file for file in os.listdir() if file.startswith("transaction") and file.endswith(".csv")]
    
    if not transaction_files:
        print("No transaction files found.")
        return

    # List to hold dataframes
    dataframes = []

    # Read each transaction file into a DataFrame and append to the list
    for file in transaction_files:
        try:
            df = pd.read_csv(file)
            dataframes.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")

    # Concatenate all dataframes into one
    if dataframes:
        aggregated_df = pd.concat(dataframes, ignore_index=True)

        # Save the aggregated dataframe to a CSV
        aggregated_df.to_csv(output_filename, index=False)
        print(f"Aggregated file saved as {output_filename}")

        # Delete the individual transaction files
        for file in transaction_files:
            try:
                os.remove(file)
                print(f"Deleted {file}")
            except Exception as e:
                print(f"Error deleting {file}: {e}")
    else:
        print("No dataframes to aggregate.")

In [8]:
def delete_files_with_prefix(prefix, extension="csv"):
    """
    Deletes all files in the current directory starting with the specified prefix 
    and having the specified extension.
    
    Args:
        prefix (str): The prefix to look for in file names.
        extension (str): The file extension to match (default is "csv").
    """
    # Get a list of all files matching the prefix and extension
    files_to_delete = [file for file in os.listdir() if file.startswith(prefix) and file.endswith(f".{extension}")]
    
    if not files_to_delete:
        #print(f"No files found with prefix '{prefix}' and extension '{extension}'.")
        return

    # Delete the matching files
    for file in files_to_delete:
        try:
            os.remove(file)
            #print(f"Deleted file: {file}")
        except Exception as e:
                continue
            #print(f"Error deleting {file}: {e}")

# Example usage
delete_files_with_prefix("transaction")

In [17]:
def aggregate_and_cleanup_csvs_1(output_filename="aggregated_transactions.csv", prefix="transaction"):
    """
    Aggregates all CSV files in the current directory that start with 'transaction'
    into one CSV file and deletes the individual files.

    Args:
        output_filename (str): The name of the output file for the aggregated data.
    
    Returns:
        dict: A dictionary with keys 'aggregated', 'deleted', and 'errors', containing:
              - 'aggregated': True if the aggregation was successful, False otherwise.
              - 'deleted': A list of files successfully deleted.
              - 'errors': A list of errors encountered during the process.
    """
    result = {
        "aggregated": False,
        "deleted": [],
        "errors": []
    }

    # Get a list of all files in the current directory starting with "transaction"
    transaction_files = [file for file in os.listdir() if file.startswith(prefix) and file.endswith(".csv")]

    if not transaction_files:
        return result

    # List to hold dataframes
    dataframes = []

    # Read each transaction file into a DataFrame and append to the list
    for file in transaction_files:
        try:
            df = pd.read_csv(file)
            dataframes.append(df)
        except Exception as e:
            result["errors"].append({"file": file, "error": str(e)})

    # Concatenate all dataframes into one
    if dataframes:
        aggregated_df = pd.concat(dataframes, ignore_index=True)

        try:
            # Save the aggregated dataframe to a CSV
            aggregated_df.to_csv(output_filename, index=False)
            result["aggregated"] = True
        except Exception as e:
            result["errors"].append({"file": output_filename, "error": str(e)})
            return result

        # Delete the individual transaction files
        #for file in transaction_files:
        #    try:
        #        os.remove(file)
        #        result["deleted"].append(file)
        #    except Exception as e:
        #        result["errors"].append({"file": file, "error": str(e)})

In [18]:
aggregate_and_cleanup_csvs_1(output_filename="uap_agg.csv", prefix="aggregated_transactions")